In [1]:
import sys; sys.path.append('..')
from osp import *

In [16]:
df_meta = get_corpus_metadata()
ids_phil = df_meta[df_meta['discipline'] == 'Philosophy'].index.tolist()
ids_lit = df_meta[df_meta['discipline'] == 'Literature'].index.tolist()

In [ ]:
from scipy.stats import fisher_exact

def fisher_test_pos(df_pos, target_col='_target', g1='Philosophy', g2='Literature'):
    """
    Run Fisher's exact test for each POS tag comparing two groups.
    
    For each POS, constructs a 2x2 contingency table:
                    | POS count | Other POS counts |
        Group 1     |    a      |        b         |
        Group 2     |    c      |        d         |
    
    Returns DataFrame with odds ratios and p-values.
    """
    # Split by group
    df_g1 = df_pos[df_pos[target_col] == g1].drop(columns=[target_col])
    df_g2 = df_pos[df_pos[target_col] == g2].drop(columns=[target_col])
    
    # Sum counts across all documents in each group
    g1_totals = df_g1.sum()
    g2_totals = df_g2.sum()
    
    # Total counts per group (across all POS)
    g1_total = g1_totals.sum()
    g2_total = g2_totals.sum()

    colname1 = f'1 ({g1})'
    colname2 = f'2 ({g2})'
    
    results = []
    for pos in g1_totals.index:
        # 2x2 contingency table
        a = g1_totals[pos]           # POS count in g1
        b = g1_total - a             # Other POS in g1
        c = g2_totals[pos]           # POS count in g2
        d = g2_total - c             # Other POS in g2
        
        table = [[a, b], [c, d]]
        odds_ratio, p_value = fisher_exact(table)
        
        results.append({
            'feat': pos,
            f'sum1': int(a),
            f'sum2': int(c),
            # f'pct{colname1}': a / g1_total * 100,
            # f'pct{colname2}': c / g2_total * 100,
            'odds_ratio': odds_ratio,
            'p_value': p_value,
            'sig': '*' if p_value < 0.05 else ('**' if p_value < 0.01 else ('***' if p_value < 0.001 else ''))
        })
    
    result_df = pd.DataFrame(results).set_index('feat')
    result_df['sig'] = result_df['p_value'].apply(lambda p: '***' if p < 0.001 else ('**' if p < 0.01 else ('*' if p < 0.05 else '')))
    return result_df.sort_values('odds_ratio', ascending=False)


In [41]:
import numpy as np

groups = [
    ('Philosophy', ids_phil),
    ('Literature', ids_lit),
]

def get_mdw_pos(ids1, ids2, name1="Group 1", name2="Group 2", feat_n=FEAT_N, feat_min_count=FEAT_MIN_COUNT, incl_deprel=True, incl_pos=True):
    df_pos_grp1 = get_pos_counts(ids1, incl_deprel=incl_deprel, incl_pos=incl_pos)
    df_pos_grp2 = get_pos_counts(ids2, incl_deprel=incl_deprel, incl_pos=incl_pos)

    words_grp1 = get_pos_word_counts(ids1)
    words_grp2 = get_pos_word_counts(ids2)

    egs_grp1 = get_pos_word_egs(ids1)
    egs_grp2 = get_pos_word_egs(ids2)


    df_pos = pd.concat([df_pos_grp1.assign(_target=name1), df_pos_grp2.assign(_target=name2)])
    fisher_results = fisher_test_pos(df_pos, target_col='_target', g1=name1, g2=name2)

    df_means = df_pos.groupby('_target').mean()
    feat2grp2mean = df_means.to_dict()
    df_sums = df_pos.groupby('_target').sum()

    ld = []
    feat1 = None

    colname1 = f'1 ({name1})'
    colname2 = f'2 ({name2})'

    for feat in feat2grp2mean:
        feat_d = {'feat':feat}
        grp2mean = feat2grp2mean[feat]
        
        feat_d[f'fpk1'] = grp2mean[name1]
        feat_d[f'fpk2'] = grp2mean[name2]
        feat_d[f'top1'] = get_egs(words_grp1[feat], n=feat_n, min_count=feat_min_count)
        feat_d[f'top2'] = get_egs(words_grp2[feat], n=feat_n, min_count=feat_min_count)

        feat_d[f'egs1'] = get_egs(words_grp1[feat], n=feat_n, min_count=feat_min_count, word2eg=egs_grp1[feat])
        feat_d[f'egs2'] = get_egs(words_grp2[feat], n=feat_n, min_count=feat_min_count, word2eg=egs_grp2[feat])
        # feat_d['total'] = feat_d[g1] + feat_d[g2]
        
        # feat1 = feat
        ld.append(feat_d)
    odf = pd.DataFrame(ld).dropna().set_index('feat')
    diffcol = f'fpk1-fpk2'
    odf[diffcol] = odf[f'fpk1'] - odf[f'fpk2']
    odf = fisher_results.join(odf).sort_values('odds_ratio', ascending=False)
    # odf['odds_ratio_log'] = np.log10(odf['odds_ratio'])
    # odf['odds_ratio_log_abs'] = np.abs(odf['odds_ratio_log'])
    odf['feat_desc'] = [POS2DESC.get(feat,'?') for feat in odf.index]
    odf = odf.reset_index()
    # odf = odf[['feat', 'feat_desc','sum1','sum2','fpk1','fpk2','fpk1-fpk2','odds_ratio','top1','top2','egs1','egs2','p_value', 'sig']]
    odf = odf.set_index(['feat','feat_desc'])
    end_cols = ['p_value', 'sig']
    odf = odf[[c for c in odf.columns if c not in end_cols] + end_cols]
    return odf.rename(columns={
        'sum1': f'# {colname1}',
        'sum2': f'# {colname2}',
        'fpk1': f'#/K {colname1}',
        'fpk2': f'#/K {colname2}',
    })


In [42]:
pd.options.display.max_colwidth = None
odf = get_mdw_pos(ids_phil, ids_lit, 'Philosophy', 'Literature', feat_n=10, feat_min_count=1, incl_deprel=False).round(2).query('sig == "***"')
odf

,,sum1 (Philosophy),sum2 (Literature),odds_ratio,#/K 1 (Philosophy),#/K 2 (Literature),top1,top2,egs1,egs2,fpk1-fpk2,p_value,sig
feat,feat_desc,,,,,,,,,,,,
NFP,Superfluous punctuation,547,71,3.91,0.33,0.08,* { ~ | - · -> > \ **,- * | { *** ;- > z ~ ^,"""G < / >(*****), we call physical"" ""Axiom P **{** ext (S) S G"" ""estimate of p (**~** I / M) is still"" ""PF(q) cF **|** EH"" ""not freeze (**-** > £ —"" ""Statistical evidence **·** Epistemic impurism"" ""for interpreting **->** as a material conditional"" ""case that y **>** xx, where xx is"" ""Iu{H) - mod? **\**, we see that"" ""Both (i ******) the stick is""","""OF MORE - AND **-** LESS : LETTER"" ""in what manner ***** Pyrrhus and f Cineas"" ""la muerte, **|** la eternajuventud"" ""impetus, hurls **{** rapii) all the"" ""Madame de L *******, of whom he has"" ""than formerly **;-** the rage of retiring"" ""alternating with **>** Leireso > or"" ""later writers. **z"" ""get? L-?n~* **~** S~"" ""STf fS, **^** Rlb^ The earth""",0.25,0.0,***
LS,List item marker,624,87,3.64,0.38,0.10,ii a i iii v k vi b iv xlvi,ii a vi v i viii iii iv xli xxxi,"""behind criterion (**ii"" ""claim above : (**a**) All the ingredients"" ""satisfaction of (**i**) is not only an"" ""motivates condition (**iii"" ""V"" ""now think of (**K** /) relative to"" ""see Section **VI**), such a search"" ""past), (**b**) it has been confirmed"" ""atypical, (**iv**) others often"" ""XLVI""","""II** fut tout puissant"" ""A**. In fact Jaggard"" ""VI"" ""V"" ""above, note **i"" ""vnI, and **vIIi**) greatly widen"" ""take effect (**iII"" ""Vandals (**IV"" ""XLI**. XLI"" ""XXXI""",0.28,0.0,***
FW,Foreign word,1553,223,3.54,0.94,0.26,i.e. e.g. etc i.e etc. e.g eds. cf pp. c.,"e.g. i.e. etc etc. pp. i.e e.g , so v.","""modern physics, **i.e.**, classical mechanics"" ""attributions, **e.g.**, mood attributions"" ""experiences, **etc"" ""inconsistent with, **i.e"" ""architecture, **etc.**, than of philosophy"" ""properties (**e.g"" ""van Koppen (**Eds.** Amsterdam : John"" ""natural language (**cf"" ""Nature and Mind, **pp.** But I shall not"" ""counterfactually to imply **C.** Fourthly, if I""","""detailed notebooks (**e.g.** Published by permission"" ""to Cleanness, **i.e.**, printing the"" ""parable, **etc"" ""another, **etc"" ""thinks Varnhagen (**pp.** But John is a shade"" ""apostrophe (**i.e"" ""the longer, **e.g"" ""el campo anima **,** tu claro verde"" ""beidenthalp bewar **so** daz ich rehte gevar"" ""Sarrazin suggests (**v.** his Kyd p. Ioo""",0.68,0.0,***
SYM,Symbol,592,120,2.51,0.36,0.14,= / + { \ /- ^ // /( x,/ = + .is,"""predicates, £ **=** { E, F,. Individual"" ""of sentences < **/** >v that are verifiable"" ""means, if K **+** is the next cardinal"" ""predicates, £ = **{** E, F,. Individual"" ""fte] |, **\** chairman\)) and"" ""difference in **/-** values of two point"" ""G t^x and x **^** t"" ""likewise acquire **//** - clauses as"" ""simply write **/(** C /) for it"" ""y) V (x **x** z) x x (y A z""","""first body [we **/** they] have any"" ""protocomes (**=** admiral or provost"" ""thus beta **+** Ν versus alpha"" ""the third.., **.is** denoted by the""",0.22,0.0,***
EX,Existential there,4933,1323,1.90,2.99,1.55,there,there,"""property, then **there** is no reason to""","""There** is still room for""",1.44,0.0,***
MD,Modal,27650,8459,1.67,16.75,9.91,can would will may must should might could shall ought,can would may will must could might should shall 'd,"""way, but it **can** be provided with"" ""specifically, he **would** observe that we"" ""The answer **will** be so context"" ""however, one **may** wonder whether"" ""holding responsible **must** answer to certain"" ""to say, and **should** have said, on"" ""While anger **might** be a good epistemic"" ""quantum physicist **could** argue that it recovers"" ""of brevity we **shall** understand the"" ""think that Nagel **ought** to drop his strong""","""the topographies **can** be understood as"" ""Holocaust victims **would** have raised objections"" ""

In [39]:
# pd.options.display.max_colwidth = None
# # pprint(dict(odf.loc['cop']))
# odf

In [6]:
ids1 = df_meta.query('discipline == "Philosophy" & 1920<=year<1970').index.tolist()
ids2 = df_meta.query('discipline == "Philosophy" & 2025>=year>=1970').index.tolist()
odf = get_mdw_pos(ids1, ids2, 'Early Philosophy', 'Late Philosophy', feat_n=10, feat_min_count=1).round(2)
odf[odf.sig=="***"]

,feat_desc,sum1,sum2,fpk1,fpk2,fpk1-fpk2,odds_ratio,sig,top1,top2,egs1,egs2,p_value
feat,,,,,,,,,,,,,
'',?,3475,5826,8.56,4.96,3.60,1.73,***,""" ' '' '. »",""" ' « »","""by definition ** ** (Nature and Mind"" ""of existence ** ** - the phrase is"" ""agent : e.g., ** ** using a motor vehicle"" ""or things **"" ""sunflowers are yellow **»** are intelligible""","""word pure ** ** when referring"" ""ext (S) ** ** S G is a partition"" ""be - doo **«** now will turn"" ""Connectives A, V, **»**. Parentheses""",0.0
``,?,3339,5760,8.22,4.91,3.32,1.68,***,""" ' « ''",""" ' » « ¬","""and about ** ** generic traits"" ""the previous ** ** vulgar measure"" ""sentences like **«** all sunflowers"" ""which remains ** ** gently optimistic""","""understand the word ** ** pure when referring"" ""fi, U and ** ** set theoretical"" ""I am uttering **»** shoo - be - doo"" ""doo - be doo **«** now , then"" ""truth to P and **¬** P but falsity to""",0.0
VBD,"Verb, past tense",2888,5501,7.11,4.69,2.43,1.52,***,was were had did thought made saw said knew became,was were had did said thought saw made took gave,"""clearer what I **was** trying to say"" ""The Greeks **were** good at it, and"" ""Dewey decided he **had** been wrong in trying"" ""that, if we **did** not say this"" ""that I once **thought** scientific ethics"" ""In them Dewey **made** a conscious attempt"" ""conditions I **saw** only one spot"" ""puzzled by what I **said** or failed to say"" ""he not only **knew**, but also wanted"" ""others, it **became** increasingly clear""","""huge literature **was** produced on this"" ""If I **were** to attempt to do"" ""procedure that **had** as its input perceptual"" ""obtain : If one **did** not utter a token"" ""she explicitly **said"" ""showing that Marx **thought** there was no such"" ""We **saw** in the previous"" ""this same work **made** thoughtful entry"" ""the referring **took** place"" ""each of which **gave** priority to its""",0.0
conj,?,15145,32596,37.30,27.76,9.54,1.35,***,have false is what more one less not has etc,have not b etc one is what has on false,"""and one should **have"" ""and colors is **false"" ""and there **is** no difference between"" ""explanation and **what** is explained"" ""conditions one or **more** positive retroflex"" ""yet he is the **one** who is responsible"" ""claims more or **less** concrete"" ""discovered or **not"" ""degree, but **has** no specifiers which"" ""wood, metals, **etc""","""want to, or **have** some evidence to"" ""conscious or **not"" ""or just before **B**) all take the"" ""experiences, **etc"" ""but not by **one** who seeks, as"" ""but not / > **is** about o"" ""and completely **what** the difference"" ""every sentence **has** both a truth value"" ""flower, and so **on"" ""at worst, **false""",0.0
CC,Coordinating conjunction,14976,32531,36.89,27.71,9.18,1.34,***,and or but both nor either yet neither,and or but both either nor yet neither & plus,"""so formidable **and** practiced a controversialist"" ""what I said **or** failed to say in"" ""by its method **but** by its subject"" ""shall contend **both** that the form of"" ""Nor** would such debate"" ""can be said **either** to be, or to lead"" ""not true, and **yet** who pcssess a language"" ""applicability, but **neither** have value or use""","""given theory T **and** classical language"" ""verifiable (**or** testable) according"" ""formal way, **but** it can be provided"" ""sentence has **both** a truth value and"" ""evolutionary forces have **either** pushed our angry"" ""experiences, **nor** to our bodily states"" ""Yet** in this case Marthas"" ""other item, **neither** to our past experiences"" ""situation (Hubin **&** Lambeth"" ""Subsective **plus** coercion""",0.0
cc,?,14721,32114,36.26,27.35,8.90,1.33,***,and or but nor yet rather as /,and or but rather / nor yet as & plus,"""that traditional **and** conventional language"" ""new distinctions **or** the new concepts"" ""my teachers, **but** by its inherent"" ""Nor** would such debate"" 

In [7]:
ids1 = df_meta.query('discipline == "Literature" & 1920<=year<1970').index.tolist()
ids2 = df_meta.query('discipline == "Literature" & 2025>=year>=1970').index.tolist()
odf = get_mdw_pos(ids1, ids2, 'Early Literature', 'Late Literature', feat_n=10, feat_min_count=1).round(2)
odf[odf.sig=="***"]

,feat_desc,sum1,sum2,fpk1,fpk2,fpk1-fpk2,odds_ratio,sig,top1,top2,egs1,egs2,p_value
feat,,,,,,,,,,,,,
csubj:pass,?,33,20,0.14,0.04,0.10,3.81,***,extension writing made come provide indicates led used superseded spelled,used done impaired establishing have threatened thing act seems promise,"""system was an **extension** of the worship"" ""Writing** and speaking should"" ""play had already **made** use of the Standards"" ""not themselves **come** in with dancing"" ""of analogy can **provide** the key to"" ""Paradise as a place **indicates** the fading in Adam"" ""themselves had **led** the way and were"" ""word is not **used** when there is a"" ""not entirely **superseded** the earlier use"" ""nearly always **spelled** in full, and that""","""cognates - are **used** extensively as"" ""done, have **done** a good, good thing"" ""individual was **impaired** in his or her"" ""Indeed **establishing** the metaphoric"" ""language, **have** a potential for"" ""society could be **threatened**, not just by rising"" ""the furthest **thing** from the minds"" ""but a simple **act** of transmission"" ""problem **seems** appropriate to"" ""is the latent **promise** of a twentieth""",0.0
LS,List item marker,40,34,0.16,0.06,0.10,2.72,***,ii a vi v i viii iii xli xxxi xi,ii vi v i a iii iv vii viii,"""second Sunday (**ii"" ""A**. In fact Jaggard"" ""VI"" ""V"" ""following : (**i**) Therefore see"" ""I. **viii**) attack on Pantalobus"" ""take effect (**iII"" ""XLI**. XLI"" ""XXXI"" ""XI""","""II**. READING THE"" ""VI"" ""V"" ""above, note **i"" ""A**. The exact relation"" ""others (v. **iii"" ""Vandals (**IV"" ""VII"" ""vnI, and **vIIi**) greatly widen""",0.0
NFP,Superfluous punctuation,36,31,0.15,0.06,0.09,2.68,***,- * ;- { | > z ~~,| - * { *** ~ ^ ') \ /.,"""Tintern Abbey : **-** If this Be but"" ""front inalte ***** rable, Contemple"" ""than formerly **;-** the rage of retiring"" ""and studies **{** Disc"" ""en castellano **|** por I A. R. y R"" ""alternating with **>** Leireso > or"" ""later writers. **z"" ""than discreet;""","""la muerte, **|** la eternajuventud"" ""OF MORE - AND **-** LESS : LETTER"" ""in what manner ***** Pyrrhus and f Cineas"" ""impetus, hurls **{** rapii) all the"" ""Madame de L *******, of whom he has"" ""get? L-?n~* **~** S~"" ""STf fS, **^** Rlb^ The earth"" ""without a dowry. **"" ""softened, in **\** Alen\con"" ""in / fiycard""",0.0
vocative,?,124,124,0.51,0.22,0.29,2.31,***,cf i sir vol op lady patkul j. god you,i sir lydia radlova anna me mutandis booth cf pauline,"""Cf**. The Duke s men"" ""I**. The principal"" ""Sir**, I am vexed"" ""Misc., **Vol"" ""Spondanus, **op"" ""Fair **lady**, do you think"" ""Now, **Patkul**, may thine injured"" ""J.** I think Milton"" ""God** he never becomes"" ""God b w i **you**, an you talk in""","""to reject (**I**) because of my"" ""Sir** you may now perceyve"" ""Lydia** I ll see you out"" ""Radlova** I ll expect you"" ""in any case, **Anna** Andreyevna"" ""Bless me, **me** too, father"" ""Mutatis **mutandis**, the Ottoman historian"" ""own account, **Booth**, the method of"" ""Cf**. The precedence"" ""Pauline** Listen, you d""",0.0
UH,Interjection,314,364,1.29,0.65,0.64,1.99,***,no oh like o well ich xli alas please nay,no yes like well oh o say um iv please,"""perhaps that **no** ( ) interpreter"" ""belief, yet, **oh"" ""secret brow, And **like** a beaten hound"" ""st not,? **O** twas a dreadful"" ""or The World **Well** Lost"" ""clear focus : **ich** darf wol guoter"" ""XLI"" ""Why, **alas**, will my young"" ""The poet should **please** both; but be sure"" ""omission of one, **nay** sometimes of two""","""annotations (**No**, Norah"" ""H.C. : **Yes**, but it means"" ""grew together, **Like** to a double cherry"" ""Well**, is it about how"" ""people share : **Oh**, if men s secret"" ""O** fieble moone"" ""and not, **say**, the natural"" ""verkiirzt oder **um** eine Geringfugigkeit"" ""IV"" ""please all, **please** none at all""",0.0
discourse,?,310,371,1.28,0.66,0.61,1.93,***,no